In [113]:

import json
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import json
from sklearn.impute import SimpleImputer
from collections import defaultdict
from collections import defaultdict

# Load and process data for pc
file_path_pc = 'bestpipelines_usc_c.json'
with open(file_path_pc, 'r') as file:
    data_pc = json.load(file)


# Load and process data for pr
file_path_pr = 'bestpipelines_usc_r.json'
with open(file_path_pr, 'r') as file:
    data_pr = json.load(file)



def get_index_order_map(data, type):
    def sort_indices_by_value_r(arr):
        # Enumerate the array: [(index, value), ...]
        """
        indexed_arr = enumerate(arr)

        # Sort the enumerated array based on the value
        sorted_arr = sorted(indexed_arr, key=lambda x: x[1], reverse=True)

        r_abbr_index_mapping = {
        "normR" : 0,
        "coR" : 1,
        "disR" : 2
        }
        index_r_abbr_mapping = {value: key for key, value in r_abbr_index_mapping.items()}


    

        # Extract the indices from the sorted array
        indices = [index_r_abbr_mapping[index] for index, value in sorted_arr]

        return indices
        
        """

        maxval = arr[0]
        maxindex = 0

        for i, val in enumerate(arr):
            if val > maxval:
                maxval = val
                maxindex = i
        return maxindex
        

    def sort_indices_by_value_c(arr):
        # Enumerate the array: [(index, value), ...]
        """
        indexed_arr = enumerate(arr)

        # Sort the enumerated array based on the value
        sorted_arr = sorted(indexed_arr, key=lambda x: x[1], reverse=True)

        c_abbr_index_mapping = {
        "normC" : 0,
        "coC" : 1,
        "disC" : 2
        }
        index_c_abbr_mapping = {value: key for key, value in c_abbr_index_mapping.items()}


    

        # Extract the indices from the sorted array
        indices = [index_c_abbr_mapping[index] for index, value in sorted_arr]

        return indices
        """
        
    
        maxval = arr[0]
        maxindex = 0

        for i, val in enumerate(arr):
            if val > maxval:
                maxval = val
                maxindex = i
        return maxindex

    indexordermap = defaultdict(list)
    
    for i, arr in enumerate(data["alpha"]):
        
        for subarr in arr:
            if type == "r":
                indexordermap[i].append(sort_indices_by_value_r(subarr))
            else:
               
                indexordermap[i].append(sort_indices_by_value_c(subarr))
    return indexordermap




import json
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import json
from sklearn.impute import SimpleImputer
from collections import defaultdict

def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))  # Recursively flatten the item if it is a list
        else:
            flattened.append(item)
    return flattened



def flatten_and_pad_list(nested_list, desired_length):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(item)
        else:
            flattened.append(item)
    
    # Pad the list if it's shorter than the desired length
    while len(flattened) < desired_length:
        flattened.append(0)
    
    return flattened

c_key_mapping = {
    "imputation.num_pc": "numerical imputation - classification",
    "imputation.cat_pc": "categorical imputation - classification",
    "nomralization.pc": "normalization - classification",  # Note: Check if 'nomralization.pc' is a typo. It might be 'normalization.pc'.
    "cleaning_outliers.pc": "cleaning outliers - classification",
    "discretization.pc": "discretization - classification",
    "pipeline.0.num_tf_prob_logits": "numerical imputation - classification",
    "pipeline.0.cat_tf_prob_logits": "categorical imputation - classification",
    "pipeline.1.tf_prob_logits": "normalization - classification",
    "pipeline.2.tf_prob_logits": "cleaning outliers - classification",
    "pipeline.3.tf_prob_logits": "discretization - classification"
   }

c_value_abbreviation_mapping = {
    "numerical imputation - classification": "nimpC",
    "categorical imputation - classification": "cimpC",
    "normalization - classification": "normC",
    "cleaning outliers - classification": "coC",
    "discretization - classification": "disC"
}


        
r_key_mapping = {
    "imputation.num_pc": "numerical imputation - regression",
    "imputation.cat_pc": "categorical imputation - regression",
    "nomralization.pc": "normalization - regression",  # Note: Check if 'nomralization.pc' is a typo. It might be 'normalization.pc'.
    "cleaning_outliers.pc": "cleaning outliers - regression",
    "discretization.pc": "discretization - regression",
    "pipeline.0.num_tf_prob_logits": "numerical imputation - regression",
    "pipeline.0.cat_tf_prob_logits": "categorical imputation - regression",
    "pipeline.1.tf_prob_logits": "normalization - regression",
    "pipeline.2.tf_prob_logits": "cleaning outliers - regression",
    "pipeline.3.tf_prob_logits": "discretization - regression"
}

r_value_abbreviation_mapping = {
    "numerical imputation - regression": "nimpR",
    "categorical imputation - regression": "cimpR",
    "normalization - regression": "normR",
    "cleaning outliers - regression": "coR",
    "discretization - regression": "disR"
}



def process_pc_data(data):
    processed_data = []
    labels = []
    point_labels = []  # New list for individual point labels
    max_length = 0
    
    # First pass to find the maximum length of sublist
    for transformations in data.values():
        for transformation in transformations:
            flat_transformation = flatten_and_pad_list(transformation, 0)  # '0' will be replaced later
            max_length = max(max_length, len(flat_transformation))
    
    # Second pass to flatten and pad data
    for transformations in data.values():
        for transformation in transformations:
            flat_transformation = flatten_and_pad_list(transformation, 0)  # '0' will be replaced later
            max_length = max(max_length, len(flat_transformation))

    
    labelarrmap = defaultdict(list)
    indexordermap = get_index_order_map(data, "c")
    # Second pass to flatten and pad data
    key_labels = []
    for key, values in data.items():
        i = 0
      
        if key == "alpha":
            continue
        for value in values:
            # Flatten the list and append
            flattened = flatten_and_pad_list(value, max_length)
            processed_data.append(flattened)
  
            if key in c_key_mapping:
                task = c_key_mapping[key]
                abreviation = c_value_abbreviation_mapping[task]
                point_labels.append("c" + str(i))
                currentorder = indexordermap[i]
                if abreviation in currentorder:
                    key_labels.append(abreviation + "_" + str(currentorder.index(abreviation)))
                    labelarrmap[str(i) + abreviation + "_" + str(currentorder.index(abreviation))].append(flattened)
                else:
                    key_labels.append("c" + abreviation)
         
            else:
                print("problem" + key)


            
            i += 1

    
    # Generate labels for each point
    # for i in range(len(processed_data)):
    #     point_labels.append(f"r{i+1}")

    return np.array(processed_data), point_labels, key_labels, labelarrmap, indexordermap


def process_pr_data(data):
    processed_data = []
    labels = []
    point_labels = []  # New list for individual point labels
    max_length = 0
    
    # First pass to find the maximum length of sublist
    for transformations in data.values():
        for transformation in transformations:
            flat_transformation = flatten_and_pad_list(transformation, 0)  # '0' will be replaced later
            max_length = max(max_length, len(flat_transformation))
    
    # Second pass to flatten and pad data
    for transformations in data.values():
        for transformation in transformations:
            flat_transformation = flatten_and_pad_list(transformation, 0)  # '0' will be replaced later
            max_length = max(max_length, len(flat_transformation))

    
    labelarrmap = {}
    indexordermap = get_index_order_map(data, "r")
    # Second pass to flatten and pad data
    key_labels = []
    for key, values in data.items():
        i = 0

        if key == "alpha":
            continue
        for value in values:
            # Flatten the list and append
            flattened = flatten_and_pad_list(value, max_length)
          
            processed_data.append(flattened)
            if key in r_key_mapping:
                task = r_key_mapping[key]
                abreviation = r_value_abbreviation_mapping[task]
                point_labels.append("r" + str(i))
                currentorder = indexordermap[i]
                if abreviation in currentorder:
                    key_labels.append(abreviation + "_" + str(currentorder.index(abreviation)))
                    labelarrmap[abreviation] = flattened
                else:
                    key_labels.append("r" + abreviation)
         
            else:
                print("problem" + key)


            
            i += 1

    
    # Generate labels for each point
    # for i in range(len(processed_data)):
    #     point_labels.append(f"r{i+1}")

    return np.array(processed_data), point_labels, key_labels, labelarrmap, indexordermap










In [114]:
X_pc, labels_pc, point_labels_pc, c_labelarrmap, c_indexordermap = process_pc_data(data_pc)
X_pr, labels_pr, point_labels_pr, r_labelarrmap, r_indexordermap = process_pr_data(data_pr)
print(r_labelarrmap)

X_combined = np.vstack((X_pc, X_pr))
labels_combined = labels_pc + labels_pr
point_labels_combined = point_labels_pc + point_labels_pr  # Combine point labels

# Run t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(X_combined)

# Create DataFrame including point labels
df_combined = pd.DataFrame(tsne_results, columns=['Dim1', 'Dim2'])
df_combined['Label'] = labels_combined
df_combined['PointLabel'] = point_labels_combined  # Add point labels to DataFrame

# Visualize using Plotly with point labels
fig = px.scatter(df_combined, x='Dim1', y='Dim2', color='Label', text='PointLabel', title='Combined t-SNE Visualization with Labels')
fig.update_traces(textposition='top center')
fig.show()

{}


In [115]:
print(c_indexordermap)
print(r_indexordermap)

defaultdict(<class 'list'>, {0: [1, 2, 2], 1: [2, 1, 0], 2: [0, 2, 0], 3: [0, 2, 1], 4: [0, 2, 2], 5: [1, 2, 2], 6: [0, 2, 0], 7: [2, 0, 1], 8: [0, 1, 0], 9: [2, 1, 0], 10: [1, 2, 1], 11: [1, 1, 0], 12: [2, 1, 0], 13: [1, 0, 0], 14: [2, 1, 0], 15: [2, 2, 0], 16: [2, 1, 0], 17: [2, 0, 0], 18: [1, 2, 1], 19: [1, 2, 0], 20: [1, 1, 0], 21: [2, 1, 0], 22: [2, 0, 1], 23: [2, 1, 0], 24: [1, 2, 2], 25: [0, 1, 2], 26: [2, 1, 0], 27: [0, 2, 2], 28: [2, 0, 2], 29: [0, 1, 2], 30: [2, 1, 0], 31: [0, 0, 0], 32: [0, 0, 2], 33: [1, 1, 1], 34: [1, 0, 0], 35: [1, 0, 1], 36: [1, 1, 0], 37: [2, 0, 0], 38: [2, 1, 0], 39: [2, 0, 1], 40: [0, 0, 1], 41: [1, 2, 2], 42: [2, 0, 2], 43: [1, 1, 0], 44: [2, 1, 1], 45: [2, 1, 0], 46: [2, 0, 0], 47: [1, 0, 2], 48: [0, 2, 1], 49: [2, 2, 1], 50: [0, 1, 1], 51: [2, 2, 0], 52: [0, 0, 0], 53: [2, 1, 1], 54: [2, 2, 1], 55: [2, 1, 0], 56: [2, 0, 0], 57: [2, 0, 1], 58: [1, 0, 2], 59: [2, 0, 0], 60: [2, 2, 0], 61: [2, 1, 2], 62: [1, 1, 0], 63: [2, 1, 0], 64: [2, 0, 1], 65: [1

In [116]:

array_of_arrays = df_combined.values.tolist()
reversed_arrays = [row[::-1] for row in array_of_arrays]


from sklearn.preprocessing import StandardScaler

def get_dim_data(dim):
    if dim == "dim1":
        dimindex = 2
    elif dim == "dim2":
        dimindex = 3
    
    dimdata = []
    for arr in reversed_arrays:
        dimdata.append(arr[dimindex])


    data = dimdata
    # Reshaping the 1D array to 2D since StandardScaler expects 2D inputs
    data_2d = np.array(data).reshape(-1, 1)

    # Creating a StandardScaler object
    scaler = StandardScaler()

    # Fitting the scaler to the data and transforming it
    normalized_data = scaler.fit_transform(data_2d)

    # Flattening the 2D array back to 1D
    normalized_data_1d = normalized_data.ravel()

    normalized_data_1d.tolist() 

    return normalized_data_1d

dim1data = get_dim_data("dim1")
dim2data = get_dim_data("dim2")

indexpipelinemap = {
    
    0 : "pipeline.1.tf_prob_logits" ,
    1 : "pipeline.2.tf_prob_logits" ,
    2 : "pipeline.3.tf_prob_logits" ,
    
}
def get_featurepipelinemap(data, indexordermap):
    featurepipelinemap = defaultdict(list)
    for i in range(len(indexordermap)):
        orderforcurrentfeature = indexordermap[i]
        for index in orderforcurrentfeature:
            index_ordermapindex = indexpipelinemap[index]
            currentarray = data[index_ordermapindex][i]
            featurepipelinemap[i].append(currentarray)
    return featurepipelinemap

c_featurepipelinemap = get_featurepipelinemap(data_pc, c_indexordermap)
r_featurepipelinemap = get_featurepipelinemap(data_pr, r_indexordermap)



label_pointlabel_dims_map = defaultdict(dict)
curindex = 0
for pointlabel, label, dim1, dim2 in reversed_arrays:
    genericpointlabel = pointlabel[1:-1]
    if "imp" in genericpointlabel:
        continue
    
    label_pointlabel_dims_map[label][genericpointlabel] = [dim1data[curindex], dim2data[curindex]]
    curindex += 1

indextransformationmap = {
    0: "norm",
    1: "co",
    2: "dis"
}



import numpy as np
from numpy.linalg import norm

def cosine_similarity(arr1, arr2):
    # Pad the shorter array with zeros
    if len(arr1) > len(arr2):
        arr2 = np.pad(arr2, (0, len(arr1) - len(arr2)), 'constant')
    elif len(arr2) > len(arr1):
        arr1 = np.pad(arr1, (0, len(arr2) - len(arr1)), 'constant')

    # Calculate cosine similarity
    dot_product = np.dot(arr1, arr2)
    norm_product = norm(arr1) * norm(arr2)
    
    if norm_product == 0:
        # Avoid division by zero
        return 0

    similarity = dot_product / norm_product

    return abs(similarity)


cosine_totaldist = 0

totcount = 0
for i in range(len(c_indexordermap)):
    clabel = "c" + str(i)
    rlabel = "r" + str(i)

    for j in range(3):
        current_c_transform = indextransformationmap[c_indexordermap[i][j]]
        current_r_transform = indextransformationmap[r_indexordermap[i][j]]

        
        current_c_array = label_pointlabel_dims_map[clabel][current_c_transform]
        current_r_array = label_pointlabel_dims_map[rlabel][current_r_transform]

        
        currentdistance = (cosine_similarity(current_c_array, current_r_array)) #take abs value?

        cosine_totaldist += currentdistance

        totcount += 1
print("total count = ", str(totcount))



import numpy as np
from scipy.interpolate import interp1d

def euclidean_distance(arr1, arr2):
    # Ensure both arrays are numpy arrays
    arr1, arr2 = np.array(arr1), np.array(arr2)
    
    # If arrays are of equal length, calculate Euclidean distance directly
    if len(arr1) == len(arr2):
        return np.linalg.norm(arr1 - arr2)

    # Interpolate the shorter array
    if len(arr1) < len(arr2):
        x = np.linspace(0, 1, len(arr1))
        f = interp1d(x, arr1, kind='linear')
        x_new = np.linspace(0, 1, len(arr2))
        arr1 = f(x_new)
    else:
        x = np.linspace(0, 1, len(arr2))
        f = interp1d(x, arr2, kind='linear')
        x_new = np.linspace(0, 1, len(arr1))
        arr2 = f(x_new)

    # Calculate Euclidean distance
    return np.linalg.norm(arr1 - arr2)

euclid_totaldist = 0

for i in range(len(c_indexordermap)):
    clabel = "c" + str(i)
    rlabel = "r" + str(i)

    for j in range(3):
        current_c_transform = indextransformationmap[c_indexordermap[i][j]]
        current_r_transform = indextransformationmap[r_indexordermap[i][j]]

        
        current_c_array = label_pointlabel_dims_map[clabel][current_c_transform]
        current_r_array = label_pointlabel_dims_map[rlabel][current_r_transform]

        
        currentdistance = (euclidean_distance(current_c_array, current_r_array)) #take abs value?

        euclid_totaldist += currentdistance




import numpy as np

def manhattan_distance(arr1, arr2):
    # Convert to numpy arrays for convenience
    arr1, arr2 = np.array(arr1), np.array(arr2)

    # Pad the shorter array with zeros
    if len(arr1) > len(arr2):
        arr2 = np.pad(arr2, (0, len(arr1) - len(arr2)), 'constant')
    elif len(arr2) > len(arr1):
        arr1 = np.pad(arr1, (0, len(arr2) - len(arr1)), 'constant')

    # Calculate Manhattan distance
    distance = np.sum(np.abs(arr1 - arr2))
    return distance
manhattan_totaldist = 0

for i in range(len(c_indexordermap)):
    clabel = "c" + str(i)
    rlabel = "r" + str(i)

    for j in range(3):
        current_c_transform = indextransformationmap[c_indexordermap[i][j]]
        current_r_transform = indextransformationmap[r_indexordermap[i][j]]

        
        current_c_array = label_pointlabel_dims_map[clabel][current_c_transform]
        current_r_array = label_pointlabel_dims_map[rlabel][current_r_transform]

        
        currentdistance = (manhattan_distance(current_c_array, current_r_array)) #take abs value?

        manhattan_totaldist += currentdistance




import numpy as np

def chebyshev_distance(arr1, arr2):
    """Compute the Chebyshev distance between two 1D numpy arrays."""
    return max(abs(a - b) for a, b in zip(arr1, arr2))


chebyshev_totaldist = 0

for i in range(len(c_indexordermap)):
    clabel = "c" + str(i)
    rlabel = "r" + str(i)

    for j in range(3):
        current_c_transform = indextransformationmap[c_indexordermap[i][j]]
        current_r_transform = indextransformationmap[r_indexordermap[i][j]]

        
        current_c_array = label_pointlabel_dims_map[clabel][current_c_transform]
        current_r_array = label_pointlabel_dims_map[rlabel][current_r_transform]

        
        currentdistance = (chebyshev_distance(current_c_array, current_r_array)) #take abs value?

        chebyshev_totaldist += currentdistance




import numpy as np

def canberra_distance(arr1, arr2):
    # Convert arrays to numpy arrays for easier manipulation
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)

    # Check if arrays are of the same length
    if arr1.shape != arr2.shape:
        raise ValueError("Arrays must be of the same length")

    # Compute the Canberra distance
    return np.sum(np.abs(arr1 - arr2) / (np.abs(arr1) + np.abs(arr2) + np.finfo(float).eps))

canberra_totaldist = 0

for i in range(len(c_indexordermap)):
    clabel = "c" + str(i)
    rlabel = "r" + str(i)

    for j in range(3):
        current_c_transform = indextransformationmap[c_indexordermap[i][j]]
        current_r_transform = indextransformationmap[r_indexordermap[i][j]]

        
        current_c_array = label_pointlabel_dims_map[clabel][current_c_transform]
        current_r_array = label_pointlabel_dims_map[rlabel][current_r_transform]

        
        currentdistance = (canberra_distance(current_c_array, current_r_array)) #take abs value?

        canberra_totaldist += currentdistance





arr = [cosine_totaldist, euclid_totaldist, manhattan_totaldist, chebyshev_totaldist, canberra_totaldist]
print(file_path_pc, file_path_pr)
print("[cosine_totaldist, euclid_totaldist, manhattan_totaldist, chebyshev_totaldist, canberra_totaldist]")
print(arr)


min_val = min(arr)
max_val = max(arr)
    
    # Avoid division by zero if all values are the same
if max_val == min_val:
    normalized = [0.5, 0.5, 0.5]  # or return the original array, as preferred
else:
    normalized = [(x - min_val) / (max_val - min_val) for x in arr]
print("normalized")
print(normalized)


total count =  321
bestpipelines_usc_c.json bestpipelines_usc_r.json
[cosine_totaldist, euclid_totaldist, manhattan_totaldist, chebyshev_totaldist, canberra_totaldist]
[205.00533450656837, 636.1036830630061, 839.0010774073471, 559.8385013898617, 522.801293865119]
normalized
[0.0, 0.67997041523338, 1.0, 0.5596775228486435, 0.5012588221878426]
